In [2]:
# Code for extracting prediction articles
# Logic (full_articles.json - articles_used_for_training)
# Author: Pandramishi Naga Sirisha
# Date: 26th May 2020

In [4]:
import pandas as pd
import urllib.request
import json 
import json
import datetime
import pytz
import dateutil.parser
from collections import defaultdict
import random
import json
import csv

In [75]:
def json_to_csv(filename, list_name, file_suffix):
    """creates a csv containing title and description of articles in a given JSON"""
    ind = 0
    title_list = []
    desc_list = []
    date_list =[]
    source_list = []
    invalid_articles = 0
    
    for json_dict in list_name:
        #check if valid title exists
        if json_dict["title"]:
            title_list.append(json_dict["title"])
            desc_list.append(json_dict["description"])
            date_list.append(dateutil.parser.parse(json_dict['publishedAt'][1]).date())
            source_list.append("CBC")
            ind += 1
            
        else:
            invalid_articles += 1
            
    print("Invalid articles :", invalid_articles)        
    prefix = filename.split(".")[0]
    out_filename = prefix  + "_" + str(ind) + "_" + file_suffix + ".csv"
    with open(out_filename, 'w') as myfile:
        writer = csv.writer(myfile)
        #wr.writerow(mylist)
        writer.writerow(["title", "description"])
        for i in range(len(title_list)):
            writer.writerow([title_list[i], desc_list[i], date_list[i], source_list[i]])
    myfile.close()
    print("Output file name is :", out_filename)
    return out_filename
                
# json_to_csv("mortgage_rates_CBC_article.json")

In [265]:
data_folder = "./Full_articles_unannotated/"

In [371]:
file_name = "housing_price_CBC_article.json"
economic_indicator_name = "housing_price"

In [372]:
economic_indicator_name

'stock_market'

In [373]:

with open(data_folder + file_name ) as f:
    list_of_json = json.load(f)

In [374]:
# def get_prediction_data(file_name):
#     """This function substracts the training data from full data and outputs prediction_articles as csv"""
#     full_article_file_name = json_to_csv(file_name, list_of_json, "convert")
#     full_df = pd.read_csv(full_article_file_name)
    
    
    
# get_prediction_data("mortgage_rates_CBC_article.json")

In [375]:
# file_name = "interest_rates_CBC_article.json"
full_article_file_name = json_to_csv(file_name, list_of_json, "convert")
full_df = pd.read_csv(full_article_file_name, sep=',', names=['title', 'description', 'publishedAt', 'source'], skiprows = 1)
full_df["title_desc"] = full_df ['title'] + ". " + full_df['description']
full_df = full_df.drop(columns = ['title', 'description'])
# full_df.set_index()
# full_df['index'] = range(0, len(full_df) )
full_df = full_df[['source','title_desc', 'publishedAt' ]]
full_df = full_df.dropna(how='any',axis=0)
full_df.head()

Invalid articles : 142
Output file name is : stock_market_CBC_article_730_convert.csv


,source,title_desc,publishedAt
0,CBC,Tesla loses $13B in market value after CEO Elo...,2020-05-01
1,CBC,How Canadians can interpret COVID-19's volatil...,2020-04-24
2,CBC,Hamilton realtor was first time stock market i...,2020-04-09
3,CBC,Manitoba securities commission warns investors...,2020-04-09
4,CBC,"Oil plunge, coronavirus fears prompt panic sel...",2020-03-09


In [384]:
import pandas
full_df['publishedAt'] =  pd.to_datetime(full_df['publishedAt'], format='%Y-%m-%d')
full_df.shape

(677, 3)

In [385]:
import pandas
cond1 = full_df.publishedAt > pd.Timestamp(2019,5,12)
x = full_df.loc[cond1, ]
x.shape

(677, 3)

In [378]:
training_df = pd.read_csv("./final_cbc_annotations/" + economic_indicator_name+ "_combined_annotations_CBC.csv", names = ['title','description', 'publishedAt', 'title_desc_sent_1', 'source'], skiprows = 1)
training_df["title_desc"] = training_df ['title'] + ". " + training_df['description']
training_df = training_df.drop(columns = ['title', 'description','title_desc_sent_1'])

training_df = training_df[['source','title_desc', 'publishedAt' ]]
training_df.iloc[0]['title_desc']

"HBC shares up on news of likely takeover deal. Baker group poised to gain control of retailer after agreeing to Catalyst Capital's higher asking price"

In [379]:
# final.shape

In [380]:
final = pd.merge(full_df,training_df,on='title_desc',how='left', indicator=True)
# final = final.dropna(how='any',axis=0)
final = final[final['_merge'] == 'left_only']
final = final.drop(columns = ['source_y','publishedAt_y','_merge'])
final.columns = ['source','title_desc', 'publishedAt']
final.shape
# final.ix[62]['title_desc'] == training_df.iloc[0]['title_desc']

(613, 3)

In [381]:
final.to_csv(file_name.split(".")[0] + "_to_predict.csv",sep=',')

In [382]:
# json_to_csv("mortgage_rates_CBC_article.json", list_of_json, "convert")

In [383]:
file_name

'stock_market_CBC_article.json'